-----
# Prepare your data for the VRNetzer 
-----

In [1]:
# TO DO : Test in Google Colab 
import networkx as nx
from sklearn import preprocessing 
import pandas as pd
import json
import random
import seaborn as sns

def normalize_xyz(d_coordinates):      
    x = [i[0] for i in d_coordinates.values()]
    y = [i[1] for i in d_coordinates.values()]
    z = [i[2] for i in d_coordinates.values()]
    xnorm = preprocessing.minmax_scale(list(x), feature_range=(0,1), axis=0, copy=True)
    ynorm = preprocessing.minmax_scale(list(y), feature_range=(0,1), axis=0, copy=True)
    znorm = preprocessing.minmax_scale(list(z), feature_range=(0,1), axis=0, copy=True)
    return xnorm,ynorm,znorm

def hex_to_rgb(hx):
    hx = hx.lstrip('#')
    hlen = len(hx)
    rgb = tuple(int(hx[i:i+hlen//3], 16) for i in range(0, hlen, hlen//3))
    return rgb

# Generate or use preprocessed Network Graph

#### RESULTS IN :
A Graph with following data structure : A nx.Graph() including node positions, node colors and node annotations stored in the nx.Graph object with the following keys : "name", "pos", "color", "annotation", "cluster", where:
- "name" stores a string i.e. title of the graph
- "pos" stores coordinates as (x,y,z)
- "nodecolor" stores hex-colors for each node, where each node has an assigned color based on your choice 
- "annotation" stores node attributes as strings, separated with a ";" For example, if nodes are persons the annotations can look like the following: name; age; height; ...- they can be as many as you like/find useful
- "cluster" stores a dictionary node: name-of-cluster 
- "linkcolor" stores a dicttionary with link ID and color assigned in hex format

In [88]:
# here an example if you have a Graph without any of the following attributes yet: 

# --------------------------
# GRAPH 
# --------------------------
G = nx.random_geometric_graph(n=20,radius=5) #nx.karate_club_graph()
if G.name == '':
    G.graph['name'] = "Testgraph"

In [89]:
# --------------------------
# POS 
# --------------------------
pos = nx.spring_layout(G, dim = 3)
xnorm,ynorm,znorm = normalize_xyz(pos)
posG = dict(zip(list(G.nodes()),zip(xnorm,ynorm,znorm)))
nx.set_node_attributes(G, posG, name="pos")

In [90]:
# --------------------------
# CLUSTER 
# --------------------------

# initial data structure : dict with key=node Index and value=assigned cluster
# replace your dictionary here and run: 
if G.name == "Zachary's Karate Club":
    dict_for_cluster = nx.get_node_attributes(G, "club")
    nx.set_node_attributes(G, dict_for_cluster, name="cluster")

#or random assigned if no assigned groups: 
else:
    assigned_groups = []
    for g in G.nodes():
        n = random.randint(0,3)
        assigned_groups.append(n)
    dict_for_cluster = dict(zip(G.nodes(), assigned_groups))
    nx.set_node_attributes(G, dict_for_cluster, name="cluster")

In [91]:
# --------------------------
# NODE COLOR  
# --------------------------
# choose and assign colors for each cluster: 
clustercolors = sns.color_palette("Set1").as_hex()[:len(set(assigned_groups))]
d_clustercolors = dict(zip(set(dict_for_cluster.values()),clustercolors)) 

# assign a clustercolor to each node based on assigned group
d_node_colors={}
for nodeid,group in dict_for_cluster.items():
    for clu, col in d_clustercolors.items():
        if group==clu:
            d_node_colors[nodeid] = col

nx.set_node_attributes(G, d_node_colors, name="nodecolor")

In [92]:
# --------------------------
# NODE ANNOTATION
# --------------------------

l_annotations = [str(i)+'; NodeID '+str(j) for i,j in zip(dict_for_cluster.values(), dict_for_cluster.keys())]
d_annotations = dict(zip(G.nodes(), l_annotations))
nx.set_node_attributes(G, d_annotations, name="annotation")

In [93]:
# --------------------------
# LINK COLOR
# --------------------------

linkcolor = '#909090'
nx.set_edge_attributes(G, linkcolor, name="linkcolor")

# TEST ATTRIBUTES of GRAPH

In [94]:
name = G.name
name

'Testgraph'

In [95]:
pos = nx.get_node_attributes(G, "pos")
pos

{0: (0.7291069754985925, 1.0, 0.506349742097395),
 1: (0.5655084946097764, 0.0, 0.30038264557162103),
 2: (0.1254511759396662, 0.7387133622859131, 0.25376432762738826),
 3: (0.2406347340692171, 0.25669283917025226, 0.14339972370050613),
 4: (0.825248178411915, 0.7358085531611424, 0.13600866325728717),
 5: (0.8205317190023307, 0.3439179137831394, 0.8942150099005464),
 6: (0.7422588497161596, 0.27826488446300635, 0.0503618062147298),
 7: (0.0, 0.44126232008135025, 0.466484095564859),
 8: (0.18910426925011442, 0.258442192436325, 0.8895001392779496),
 9: (0.49461890283057514, 0.49545751237451796, 1.0),
 10: (0.933284965180674, 0.7066260529052308, 0.7279446048627678),
 11: (0.4326187262733394, 0.5486371238527439, 0.0),
 12: (0.2981372061183615, 0.9604870955357931, 0.5780624631090735),
 13: (1.0, 0.4811981029117915, 0.3868889201783611),
 14: (0.4717849891615238, 0.9154817257091732, 0.20270515544906997),
 15: (0.11825582143678975, 0.6858476462122404, 0.8021047045736527),
 16: (0.2030985145606

In [96]:
nodecolor = nx.get_node_attributes(G, "nodecolor")
nodecolor

{0: '#4daf4a',
 1: '#4daf4a',
 2: '#e41a1c',
 3: '#984ea3',
 4: '#e41a1c',
 5: '#984ea3',
 6: '#e41a1c',
 7: '#e41a1c',
 8: '#984ea3',
 9: '#377eb8',
 10: '#e41a1c',
 11: '#4daf4a',
 12: '#377eb8',
 13: '#984ea3',
 14: '#377eb8',
 15: '#377eb8',
 16: '#984ea3',
 17: '#4daf4a',
 18: '#984ea3',
 19: '#984ea3'}

In [97]:
cluster = nx.get_node_attributes(G, "cluster")
cluster

{0: 2,
 1: 2,
 2: 0,
 3: 3,
 4: 0,
 5: 3,
 6: 0,
 7: 0,
 8: 3,
 9: 1,
 10: 0,
 11: 2,
 12: 1,
 13: 3,
 14: 1,
 15: 1,
 16: 3,
 17: 2,
 18: 3,
 19: 3}

In [98]:
annotation = nx.get_node_attributes(G, "annotation")
annotation

{0: '2; NodeID 0',
 1: '2; NodeID 1',
 2: '0; NodeID 2',
 3: '3; NodeID 3',
 4: '0; NodeID 4',
 5: '3; NodeID 5',
 6: '0; NodeID 6',
 7: '0; NodeID 7',
 8: '3; NodeID 8',
 9: '1; NodeID 9',
 10: '0; NodeID 10',
 11: '2; NodeID 11',
 12: '1; NodeID 12',
 13: '3; NodeID 13',
 14: '1; NodeID 14',
 15: '1; NodeID 15',
 16: '3; NodeID 16',
 17: '2; NodeID 17',
 18: '3; NodeID 18',
 19: '3; NodeID 19'}

In [99]:
linkcolor = nx.get_edge_attributes(G, "linkcolor")
linkcolor

{(0, 1): '#909090',
 (0, 2): '#909090',
 (0, 3): '#909090',
 (0, 4): '#909090',
 (0, 5): '#909090',
 (0, 6): '#909090',
 (0, 7): '#909090',
 (0, 8): '#909090',
 (0, 9): '#909090',
 (0, 10): '#909090',
 (0, 11): '#909090',
 (0, 12): '#909090',
 (0, 13): '#909090',
 (0, 14): '#909090',
 (0, 15): '#909090',
 (0, 16): '#909090',
 (0, 17): '#909090',
 (0, 18): '#909090',
 (0, 19): '#909090',
 (1, 2): '#909090',
 (1, 3): '#909090',
 (1, 4): '#909090',
 (1, 5): '#909090',
 (1, 6): '#909090',
 (1, 7): '#909090',
 (1, 8): '#909090',
 (1, 9): '#909090',
 (1, 10): '#909090',
 (1, 11): '#909090',
 (1, 12): '#909090',
 (1, 13): '#909090',
 (1, 14): '#909090',
 (1, 15): '#909090',
 (1, 16): '#909090',
 (1, 17): '#909090',
 (1, 18): '#909090',
 (1, 19): '#909090',
 (2, 3): '#909090',
 (2, 4): '#909090',
 (2, 5): '#909090',
 (2, 6): '#909090',
 (2, 7): '#909090',
 (2, 8): '#909090',
 (2, 9): '#909090',
 (2, 10): '#909090',
 (2, 11): '#909090',
 (2, 12): '#909090',
 (2, 13): '#909090',
 (2, 14): '#9090

# UPLOADING DATA PREPARATION:
There are the following options to prepare your data for the VRnetzer
* 1 - using JSON files
* 2 - using CSV files


### OPTION 1: GENERATE GRAPH upload / JSON file 

In [100]:
G_json = json.dumps(nx.node_link_data(G))

with open(G.name+".json", "w") as outfile:
    outfile.write(G_json)

### OPTION 2: GENERATE CSV files 

In [101]:
# NODE POSITIONS 

df_nodepos = pd.DataFrame()
df_nodepos['x']=[i[0] for i in posG.values()]
df_nodepos['y']=[i[1] for i in posG.values()]
df_nodepos['z']=[i[2] for i in posG.values()]

df_nodepos.to_csv('output/VRNetzer2_ExamplesFiles/'+G.name+'_nodepositions.csv', header=None, index=0)

In [102]:
# NODE COLORS 

df_nodecol = pd.DataFrame()
df_nodecol['r']=[hex_to_rgb(i)[0] for i in d_node_colors.values()]
df_nodecol['g']=[hex_to_rgb(i)[1] for i in d_node_colors.values()]
df_nodecol['b']=[hex_to_rgb(i)[2] for i in d_node_colors.values()]
df_nodecol['a']=[100 for i in d_node_colors.values()]

df_nodecol.to_csv('output/VRNetzer2_ExamplesFiles/'+G.name+'_nodecolors.csv', header=None, index=0)

In [103]:
# NODE PROPERTIES

df_nodeprop = pd.DataFrame()
df_nodeprop['prop'] = l_annotations

df_nodeprop.to_csv('output/VRNetzer2_ExamplesFiles/'+G.name+'_nodeproperties.csv', header=None, index=0)

In [104]:
# LINKS

df_links = pd.DataFrame()
df_links['start'] = [i[0] for i in list(G.edges())]
df_links['end'] = [i[1] for i in list(G.edges())]

df_links.to_csv('output/VRNetzer2_ExamplesFiles/'+G.name+'_links.csv', header=None, index=0)

In [105]:
# LINK COLORS

df_linkcol = pd.DataFrame()
df_linkcol['r'] = [hex_to_rgb(i)[0] for i in linkcolor.values()] #hex_to_rgb(linkcolor)[0] 
df_linkcol['g'] = [hex_to_rgb(i)[1] for i in linkcolor.values()] #hex_to_rgb(linkcolor)[1] 
df_linkcol['b'] = [hex_to_rgb(i)[2] for i in linkcolor.values()] #hex_to_rgb(linkcolor)[2] 
df_linkcol['a'] = 80

df_linkcol.to_csv('output/VRNetzer2_ExamplesFiles/'+G.name+'_linkcolors.csv', header=None, index=0)

In [106]:
# CLUSTER LABELS

clusterlabels = []
for cluster in d_clustercolors.keys():
    sublist = []
    for k,v in dict_for_cluster.items():
        if cluster == v:
            sublist.append(str(k))
    sublist.insert(0,str(cluster))
    clusterlabels.append(sublist)

df_labels = pd.DataFrame(clusterlabels)

df_labels.to_csv('output/VRNetzer2_ExamplesFiles/'+G.name+'_clusterlabels.csv', header=None, index=0)